In [ ]:
import pandas as pd
import numpy as np
import os
import glob
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import gc
import math
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Capacity Grouping

In [ ]:
## Get Info about the WTP Capacity and water source
wtpc = pd.read_csv('Design Capacity WTP.csv')
wlb = ['small','medium','large']
breaks = [0,20,200,np.inf]
wtpc['ng'] = pd.cut(x=wtpc['Design Capacity (MLD)'], bins=breaks,labels=wlb)
wtpc['nng'] = pd.cut(x=wtpc['Design Capacity (MLD)'], bins=breaks)

## General Data Loading & Cleaning

### Water Quality (WQ) Related Data 

In [ ]:
path = r'WQ\Refined'
db_dir = glob.glob(path + "/*.csv")

li = []

## Joining all csv file within the folder
for i in db_dir:
    df = pd.read_csv(i, index_col=None, header=0)
    print(i)
    li.append(df)
fdb = pd.concat(li, ignore_index=True)

WQ\Refined\WQDB_APR_21.csv
WQ\Refined\WQDB_Aug_21.csv
WQ\Refined\WQDB_FEB_21.csv
WQ\Refined\WQDB_Jan_21.csv
WQ\Refined\WQDB_Jul_21.csv
WQ\Refined\WQDB_June_21.csv
WQ\Refined\WQDB_MAR_21.csv
WQ\Refined\WQDB_May_21.csv
WQ\Refined\WQDB_NOV_21.csv
WQ\Refined\WQDB_Oct_21.csv
WQ\Refined\WQDB_Sept_21.csv
WQ\Refined\WQ_2022_Data_All.csv


In [ ]:
## Drop unwanted column and fill na with big number 9999, then it will be removed when there is a change in data
fdb = fdb.drop(['Unnamed: 0'],axis=1)
fdb['Date'] = pd.to_datetime(fdb['Date'])
cl = fdb.loc[:, fdb.dtypes == object].columns.to_list()
cl.remove('wtp')
for i in cl:
    fdb[i] = fdb[i].fillna(9999)
    fdb[i] = fdb[i].apply(lambda x: str(x).replace(',',''))
fdb[cl] = fdb[cl].apply(pd.to_numeric)
db = fdb.copy()

c:\Users\AIS06077\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '13/10/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
c:\Users\AIS06077\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '14/10/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
c:\Users\AIS06077\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '15/10/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
c:\Users\AIS06077\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '16/10/2022' in DD/MM/YYYY format. Provide form

In [ ]:
## Get all the relevant columns (Daily average for further processing)
avg = fdb.columns
l = ['Quarter','wtp','Month','Date']
avg = [x for x in avg if "avg" in x ] + l
db['Month'] = db['Date'].dt.month
db['Quarter'] = db['Date'].dt.quarter
dc = db[avg].copy()

## Drop due to data limitation (Only limited number of WTP do it on daily basis)
dc= dc.drop(['Aluminium_avg', 'Fluoride_avg'],axis=1)
avc = [x for x in dc.columns if "avg" in x ]

In [ ]:
## Final Clean data for further processing
dc = pd.merge(dc,wtpc[['wtp','Water Source','Lembangan']],on='wtp',how='left')
dc = dc[(dc['Lembangan'].isin(['Sg Selangor'])) & (dc['Water Source']=='River')].copy()
dc = dc[dc['pH_avg']<14].copy()

In [ ]:
dc.columns

Index(['pH_avg', 'Turbidity_avg', 'Color_avg', 'Iron_avg', 'Ammonia_avg',
       'Manganese_avg', 'Quarter', 'wtp', 'Month', 'Date', 'Water Source',
       'Lembangan'],
      dtype='object')

## Chemical Dosage

#### Overall DB

In [ ]:
path = r'CC\Chm_DB'
dcb_dir = glob.glob(path + "/*.csv")

mm = []

for i in dcb_dir:
    dbo = pd.read_csv(i, index_col=None, header=0)
    print(i)
    
    mm.append(dbo)

CC\Chm_DB\CHEMI_FDB_21.csv
CC\Chm_DB\CHEMI_FDB_22.csv
CC\Chm_DB\CHEMI_SP_V2.csv


In [ ]:
chdb = pd.concat(mm, ignore_index=True)
chdb = chdb.drop(['Unnamed: 0'],axis=1)
chdb = chdb[chdb['Date']!='MONTHLY'].reset_index(drop=True)
#chdb['Date'] = chdb['Date'] + '-2021'
chdb.to_csv('chdb.csv')
chdb['Date'] = pd.to_datetime(chdb['Date'])
chdb['Month'] = chdb['Date'].dt.month
chdb['Quarter'] = chdb['Date'].dt.quarter

c:\Users\AIS06077\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '13/11/2021' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
c:\Users\AIS06077\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '14/11/2021' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
c:\Users\AIS06077\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '15/11/2021' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
c:\Users\AIS06077\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '16/11/2021' in DD/MM/YYYY format. Provide form

In [ ]:
print(len(chdb.columns))

149


In [ ]:
FC = chdb.columns
fFC = [x for x in FC if "avg" in x]
rcol = ['wtp','Date','Month','Quarter']
xxdb = chdb[fFC+rcol].copy()

In [ ]:
new = [x for x in FC if "avg" in x]
new = [x.replace(" _avg","") for x in new]
new = [x.replace("_avg","") for x in new]

### Regrouping Chemical Product to it's own purposes

In [ ]:
xdic = dict(zip(fFC,new))
xxdb = xxdb.rename(columns=xdic)
xxdb = pd.merge(xxdb,wtpc[['wtp','Water Source','Lembangan']],on='wtp',how='left')
x1db = xxdb[(xxdb['Lembangan']=='Sg Selangor') & (xxdb['Water Source']=='River')].copy()

In [ ]:
my_list = list(chdb.columns)
duplicates = set([x for x in my_list if my_list.count(x) > 1])
duplicates

set()

In [ ]:
appl = x1db.groupby(['wtp']).mean().reset_index()
wtpn = appl.wtp.unique()
number = dict({'wtp':[],'Chem':[]})
for i in wtpn:
    number['wtp'].append(i)
    jde = appl[appl['wtp']==i].drop(['wtp'],axis=1)
    jde = jde.dropna(axis=1).columns
    number['Chem'].append(jde)
cgr = pd.read_csv("C.Grouping.csv")
Chdict = dict(zip(cgr['Chemical Name'].values,cgr['Correct Grouping']))
df22=pd.DataFrame(number)
df22=df22.assign(Chem=df22.Chem.map(','.join))
df22=df22.set_index(['wtp']).apply(lambda x: x.str.split(',').explode()).reset_index()
df22['Purpose']=df22['Chem'].apply(lambda x: Chdict.get(x))

In [ ]:
lis = df22['Chem'].unique()
pre_lis = [x for x in lis if "Pre" in x]
pos_lis = [x for x in lis if "Post" in x]
int_lis = [x for x in lis if "Intermediate" in x]
total_lis = pre_lis + pos_lis + int_lis

In [ ]:
sd = ['Soda Ash (Pre)','Soda Ash (Post)']
hl = ['Hydrated Lime (Powder) (Pre)','Hydrated Lime (Powder) (Post)','Hydrated Lime (Pre) (mg/L)','Hydrated Lime (Post) (mg/L)']
lc = ['Liquid Chlorine (Pre)','Liquid Chlorine (Post)','Liquid Chlorine (Pre) (mg/L)','Liquid Chlorine (Post) (mg/L)']

In [ ]:
x1db.to_csv('x1db.csv')

In [ ]:
my_list = list(x1db.columns)

In [ ]:
duplicates = set([x for x in my_list if my_list.count(x) > 1])
duplicates

set()

In [ ]:
x1db.columns

Index(['Kibbled Aluminium Sulphate', 'Flopam FA 920 PWG', 'Soda Ash (Pre)',
       'Soda Ash (Post)', 'Liquid Chlorine', 'Sodium Silico Fluoride',
       'Liquid Aluminium Sulphate', 'Hydrated Lime (Pre)',
       'Hydrated Lime (Post)', 'Soda Ash', 'Hydrated Lime',
       'Liquid Polyaluminium Chloride', 'Liquid Chlorine (Pre)',
       'Liquid Chlorine (Intermediate)', 'Liquid Chlorine (Post)',
       'Sodium Aluminosilicate', 'Sodium Fluoride', 'Superfloc C 591 PWG',
       'Aluminium Chlorohydrate', 'Flopam AH 912 PWG',
       'Hydrated Lime (Intermediate)', 'Hydrated Lime (Powder) (Pre)',
       'Hydrated Lime (Powder) (Post)', 'Hydrated Lime (Powder)',
       'Flopam FO 4190 PWG', 'Flopam FO 4290 PWG',
       'Polyelectrolyte AN 910 PWG', 'Microsand', 'Activated Carbon',
       'Flopam FL 4520 PWG', 'Modified Liquid Alum', 'Potassium Permanganate',
       'Polymer Tianfloc C3080', 'Sodium Hypochlorite',
       'Hydrated Lime (Powder) (Intermediate)',
       'Liquid Polyaluminium Ch

In [ ]:
x1db[total_lis] = x1db[total_lis].fillna(0)
x1db['Soda_Ash_T'] = x1db[sd].sum(axis=1)
x1db['Hydrated_Lime_Pow_T'] = x1db[hl].sum(axis=1)
x1db['Liquid_Chlorine_T'] = x1db[lc].sum(axis=1)

In [ ]:
x1db.columns

Index(['Kibbled Aluminium Sulphate', 'Flopam FA 920 PWG', 'Soda Ash (Pre)',
       'Soda Ash (Post)', 'Liquid Chlorine', 'Sodium Silico Fluoride',
       'Liquid Aluminium Sulphate', 'Hydrated Lime (Pre)',
       'Hydrated Lime (Post)', 'Soda Ash', 'Hydrated Lime',
       'Liquid Polyaluminium Chloride', 'Liquid Chlorine (Pre)',
       'Liquid Chlorine (Intermediate)', 'Liquid Chlorine (Post)',
       'Sodium Aluminosilicate', 'Sodium Fluoride', 'Superfloc C 591 PWG',
       'Aluminium Chlorohydrate', 'Flopam AH 912 PWG',
       'Hydrated Lime (Intermediate)', 'Hydrated Lime (Powder) (Pre)',
       'Hydrated Lime (Powder) (Post)', 'Hydrated Lime (Powder)',
       'Flopam FO 4190 PWG', 'Flopam FO 4290 PWG',
       'Polyelectrolyte AN 910 PWG', 'Microsand', 'Activated Carbon',
       'Flopam FL 4520 PWG', 'Modified Liquid Alum', 'Potassium Permanganate',
       'Polymer Tianfloc C3080', 'Sodium Hypochlorite',
       'Hydrated Lime (Powder) (Intermediate)',
       'Liquid Polyaluminium Ch

In [ ]:
len(x1db[total_lis].fillna(0).columns)

10

In [ ]:
li = [dict(zip(sd,['Soda_Ash_T' for i in range(len(sd))])),
      dict(zip(lc,['Liquid_Chlorine_T' for i in range(len(lc))])),
      dict(zip(hl,['Hydrated_Lime_Pow_T' for i in range(len(hl))]))]

In [ ]:
d = {}
for dictionary in li:
    for k, v in dictionary.items():
        d[k] = v

In [ ]:
df22['Chem'] = df22['Chem'].apply(lambda x: d.get(x,x))
df22 = df22.drop_duplicates(subset=['wtp','Chem'])
df22 = df22.drop_duplicates(subset=['wtp','Purpose'],keep='first')
tt = x1db['wtp'].unique()
trdf = df22.copy()
Chdict['Hydrated_Lime_Pow_T'] = 'pH Adjuster'
Chdict['Soda_Ash_T'] = 'pH Adjuster'
Chdict['Liquid_Chlorine_T'] = 'Disinfectant'

In [ ]:
jam = pd.DataFrame()
x10db = x1db.copy()

In [322]:
x10db

,Kibbled Aluminium Sulphate,Flopam FA 920 PWG,Soda Ash (Pre),Soda Ash (Post),Liquid Chlorine,Sodium Silico Fluoride,Liquid Aluminium Sulphate,Hydrated Lime (Pre),Hydrated Lime (Post),Soda Ash,Hydrated Lime,Liquid Polyaluminium Chloride,Liquid Chlorine (Pre),Liquid Chlorine (Intermediate),Liquid Chlorine (Post),Sodium Aluminosilicate,Sodium Fluoride,Superfloc C 591 PWG,Aluminium Chlorohydrate,Flopam AH 912 PWG,Hydrated Lime (Intermediate),Hydrated Lime (Powder) (Pre),Hydrated Lime (Powder) (Post),Hydrated Lime (Powder),Flopam FO 4190 PWG,Flopam FO 4290 PWG,Polyelectrolyte AN 910 PWG,Microsand,Activated Carbon,Flopam FL 4520 PWG,Modified Liquid Alum,Potassium Permanganate,Polymer Tianfloc C3080,Sodium Hypochlorite,Hydrated Lime (Powder) (Intermediate),Liquid Polyaluminium Chloride,Liquid Polyaluminium Chloride (mg/L),Raw Water Polyelectrolyte 4290 PWG / 4190 PWG / AN910 (mg/L),Hydrated Lime (Post) (mg/L),Liquid Chlorine (Post) (mg/L),Sodium Silico Fluoride (mg/L),Liquid Chlorine (Pre) (mg/L),Hydrated Lime (Pre) (mg/L),wtp,Date,Month,Quarter,Water Source,Lembangan,Soda_Ash_T,Hydrated_Lime_Pow_T,Liquid_Chlorine_T
0,14.23,0.02,1.05,0.0,2.38,0.46,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,0.000000,0.0,Batang Kali,2021-01-04,1,1,River,Sg Selangor,1.05,0.000000,0.000000
1,13.33,0.02,1.01,0.0,2.36,0.46,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,0.000000,0.0,Batang Kali,2021-02-04,2,1,River,Sg Selangor,1.01,0.000000,0.000000
2,16.47,0.02,0.98,0.0,2.26,0.44,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,0.000000,0.0,Batang Kali,2021-03-04,3,1,River,Sg Selangor,0.98,0.000000,0.000000
3,16.32,0.02,1.03,0.0,2.36,0.46,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,0.000000,0.0,Batang Kali,2021-04-04,4,2,River,Sg Selangor,1.03,0.000000,0.000000
4,17.09,0.02,1.05,0.0,2.35,0.47,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,0.000000,0.0,Batang Kali,2021-05-04,5,2,River,Sg Selangor,1.05,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25125,NaN,NaN,0.00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.230479,0.048502,12.399601,5.015964,0.578928,1.189927,0.0,Sungai Selangor Fasa 1,2021-12-27,12,4,River,Sg Selangor,0.00,12.399601,6.205891
25126,NaN,NaN,0.00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.957065,0.082025,12.185244,5.067093,0.781969,1.308897,0.0,Sungai Selangor Fasa 1,2021-12-28,12,4,River,Sg Selangor,0.00,12.185244,6.375990
25127,NaN,NaN,0.00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.296242,0.048745,9.433292,4.935568,0.686752,1.151963,0.0,Sungai Selangor Fasa 1,2021-12-29,12,4,River,Sg Selangor,0.00,9.433292,6.087530
25128,NaN,NaN,0.00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.994393,0.048250,17.753680,4.955404,0.745078,1.180927,0.0,Sungai Selangor Fasa 1,2021-12-30,12,4,River,Sg Selangor,0.00,17.753680,6.136331


In [323]:
#x10db['Liquid Polyaluminium Chloride'] = x10db['Liquid Polyaluminium Chloride'] + x10db['Liquid Polyaluminium Chloride .1']
#trdf = trdf.drop(trdf[trdf['Chem'] == 'Liquid Polyaluminium Chloride .1'].index)
#x10db = x10db.drop('Liquid Polyaluminium Chloride .1',axis = 1)
x10db.to_csv('CP_data\ChemPare_SgSel_2_2022.csv')

In [324]:
print(trdf)

                                wtp  \
0                       Batang Kali   
1                       Batang Kali   
2                       Batang Kali   
4                       Batang Kali   
5                       Batang Kali   
6                       Batang Kali   
8                  Kuala Kubu Bharu   
9                  Kuala Kubu Bharu   
10                 Kuala Kubu Bharu   
11                 Kuala Kubu Bharu   
13                 Kuala Kubu Bharu   
15               Rantau Panjang New   
16               Rantau Panjang New   
17               Rantau Panjang New   
18               Rantau Panjang New   
19               Rantau Panjang New   
20               Rantau Panjang New   
23               Rantau Panjang Old   
24               Rantau Panjang Old   
25               Rantau Panjang Old   
26               Rantau Panjang Old   
27               Rantau Panjang Old   
28               Rantau Panjang Old   
30                             Rasa   
31                       

In [325]:
Chdict_2 = {}

for key in Chdict:
    Chdict_2[key] = key

In [326]:
x10db.columns

Index(['Kibbled Aluminium Sulphate', 'Flopam FA 920 PWG', 'Soda Ash (Pre)',
       'Soda Ash (Post)', 'Liquid Chlorine', 'Sodium Silico Fluoride',
       'Liquid Aluminium Sulphate', 'Hydrated Lime (Pre)',
       'Hydrated Lime (Post)', 'Soda Ash', 'Hydrated Lime',
       'Liquid Polyaluminium Chloride', 'Liquid Chlorine (Pre)',
       'Liquid Chlorine (Intermediate)', 'Liquid Chlorine (Post)',
       'Sodium Aluminosilicate', 'Sodium Fluoride', 'Superfloc C 591 PWG',
       'Aluminium Chlorohydrate', 'Flopam AH 912 PWG',
       'Hydrated Lime (Intermediate)', 'Hydrated Lime (Powder) (Pre)',
       'Hydrated Lime (Powder) (Post)', 'Hydrated Lime (Powder)',
       'Flopam FO 4190 PWG', 'Flopam FO 4290 PWG',
       'Polyelectrolyte AN 910 PWG', 'Microsand', 'Activated Carbon',
       'Flopam FL 4520 PWG', 'Modified Liquid Alum', 'Potassium Permanganate',
       'Polymer Tianfloc C3080', 'Sodium Hypochlorite',
       'Hydrated Lime (Powder) (Intermediate)',
       'Liquid Polyaluminium Ch

In [327]:
trdf[trdf['wtp']==i]['Chem'].unique()

array(['Liquid Polyaluminium Chloride (mg/L)',
       'Raw Water Polyelectrolyte 4290 PWG / 4190 PWG / AN910 (mg/L)',
       'Hydrated_Lime_Pow_T', 'Liquid_Chlorine_T',
       'Sodium Silico Fluoride (mg/L)', 'Month'], dtype=object)

#### Chemical Product Normalization

In [328]:
onew = 0
for i in tt:
    
    c2t = trdf[trdf['wtp']==i]['Chem'].unique()
    c2t = np.append(c2t,np.array(['Date','wtp']))
    c2t
    op = x10db[x10db['wtp']==i][c2t].copy()
    op = op.rename(columns=Chdict_2)
    
    if onew == 0:
        jam = jam.append(op)
    else:
        jam = pd.concat([jam, op], ignore_index=False)
    
    onew = onew+1
    
    del c2t, op
    gc.collect()

array(['Kibbled Aluminium Sulphate', 'Flopam FA 920 PWG', 'Soda_Ash_T',
       'Liquid Chlorine', 'Sodium Silico Fluoride', 'Month', 'Date',
       'wtp'], dtype=object)

C:\Users\AIS06077\AppData\Local\Temp\ipykernel_4716\3742793569.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  jam = jam.append(op)


216

array(['Kibbled Aluminium Sulphate', 'Liquid Chlorine',
       'Sodium Silico Fluoride', 'Soda Ash', 'Month', 'Date', 'wtp'],
      dtype=object)

0

...

0

array(['Liquid Polyaluminium Chloride (mg/L)',
       'Raw Water Polyelectrolyte 4290 PWG / 4190 PWG / AN910 (mg/L)',
       'Hydrated_Lime_Pow_T', 'Liquid_Chlorine_T',
       'Sodium Silico Fluoride (mg/L)', 'Month', 'Date', 'wtp'],
      dtype=object)

0

array(['Liquid Polyaluminium Chloride (mg/L)',
       'Raw Water Polyelectrolyte 4290 PWG / 4190 PWG / AN910 (mg/L)',
       'Hydrated_Lime_Pow_T', 'Liquid_Chlorine_T',
       'Sodium Silico Fluoride (mg/L)', 'Month', 'Date', 'wtp'],
      dtype=object)

0

In [329]:
jam

,Kibbled Aluminium Sulphate,Flopam FA 920 PWG,Soda_Ash_T,Liquid Chlorine,Sodium Silico Fluoride,Month,Date,wtp,Soda Ash,Hydrated Lime,Liquid Polyaluminium Chloride,Sodium Aluminosilicate,Liquid Polyaluminium Chloride,Liquid Aluminium Sulphate,Liquid_Chlorine_T,Hydrated_Lime_Pow_T,Flopam FO 4290 PWG,Microsand,Liquid Polyaluminium Chloride (mg/L),Raw Water Polyelectrolyte 4290 PWG / 4190 PWG / AN910 (mg/L),Sodium Silico Fluoride (mg/L)
0,14.23,0.02,1.05,2.38,0.46,1.0,2021-01-04,Batang Kali,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13.33,0.02,1.01,2.36,0.46,2.0,2021-02-04,Batang Kali,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16.47,0.02,0.98,2.26,0.44,3.0,2021-03-04,Batang Kali,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16.32,0.02,1.03,2.36,0.46,4.0,2021-04-04,Batang Kali,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,17.09,0.02,1.05,2.35,0.47,5.0,2021-05-04,Batang Kali,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25125,NaN,NaN,NaN,NaN,NaN,12.0,2021-12-27,Sungai Selangor Fasa 1,NaN,NaN,NaN,NaN,NaN,NaN,6.205891,12.399601,NaN,NaN,37.230479,0.048502,0.578928
25126,NaN,NaN,NaN,NaN,NaN,12.0,2021-12-28,Sungai Selangor Fasa 1,NaN,NaN,NaN,NaN,NaN,NaN,6.375990,12.185244,NaN,NaN,37.957065,0.082025,0.781969
25127,NaN,NaN,NaN,NaN,NaN,12.0,2021-12-29,Sungai Selangor Fasa 1,NaN,NaN,NaN,NaN,NaN,NaN,6.087530,9.433292,NaN,NaN,30.296242,0.048745,0.686752
25128,NaN,NaN,NaN,NaN,NaN,12.0,2021-12-30,Sungai Selangor Fasa 1,NaN,NaN,NaN,NaN,NaN,NaN,6.136331,17.753680,NaN,NaN,30.994393,0.048250,0.745078


### Final Merge Data

In [330]:
wqdb1 = dc[dc['Lembangan']=='Sg Selangor'][['wtp', 'pH_avg', 'Date', 'Turbidity_avg', 'Color_avg',
       'Iron_avg', 'Ammonia_avg', 'Manganese_avg']].copy()

## Dropping Polyelectrolyte because project does not focus study on these product (Low usage & only certain WTP using it)
FFDB = pd.merge(wqdb1.copy(),jam.copy(),on=['wtp','Date'],how='left')


In [331]:
FFDB.columns

Index(['wtp', 'pH_avg', 'Date', 'Turbidity_avg', 'Color_avg', 'Iron_avg',
       'Ammonia_avg', 'Manganese_avg', 'Kibbled Aluminium Sulphate',
       'Flopam FA 920 PWG', 'Soda_Ash_T', 'Liquid Chlorine',
       'Sodium Silico Fluoride', 'Month', 'Soda Ash', 'Hydrated Lime',
       'Liquid Polyaluminium Chloride', 'Sodium Aluminosilicate',
       'Liquid Polyaluminium Chloride ', 'Liquid Aluminium Sulphate',
       'Liquid_Chlorine_T', 'Hydrated_Lime_Pow_T', 'Flopam FO 4290 PWG',
       'Microsand', 'Liquid Polyaluminium Chloride (mg/L)',
       'Raw Water Polyelectrolyte 4290 PWG / 4190 PWG / AN910 (mg/L)',
       'Sodium Silico Fluoride (mg/L)'],
      dtype='object')

In [332]:
FFDB.to_csv('CP_data\ChemPare_SgSel_2_2022.csv')
jam4 = FFDB.copy().fillna(0)

In [333]:
FFDB.columns

Index(['wtp', 'pH_avg', 'Date', 'Turbidity_avg', 'Color_avg', 'Iron_avg',
       'Ammonia_avg', 'Manganese_avg', 'Kibbled Aluminium Sulphate',
       'Flopam FA 920 PWG', 'Soda_Ash_T', 'Liquid Chlorine',
       'Sodium Silico Fluoride', 'Month', 'Soda Ash', 'Hydrated Lime',
       'Liquid Polyaluminium Chloride', 'Sodium Aluminosilicate',
       'Liquid Polyaluminium Chloride ', 'Liquid Aluminium Sulphate',
       'Liquid_Chlorine_T', 'Hydrated_Lime_Pow_T', 'Flopam FO 4290 PWG',
       'Microsand', 'Liquid Polyaluminium Chloride (mg/L)',
       'Raw Water Polyelectrolyte 4290 PWG / 4190 PWG / AN910 (mg/L)',
       'Sodium Silico Fluoride (mg/L)'],
      dtype='object')

In [385]:
jam4['Soda_Ash'] = jam4.apply(lambda row: row['Soda_Ash_T'] + row['Soda Ash'], axis=1)
jam4['Liquid_Chlorine'] = jam4.apply(lambda row: row['Liquid_Chlorine_T'] + row['Liquid Chlorine'], axis=1)
jam4['Liquid_Polyaluminium_Chloride'] = jam4.apply(lambda row: row['Liquid Polyaluminium Chloride (mg/L)'] + row['Liquid Polyaluminium Chloride'], axis=1)
jam4['Hydrated_Lime'] = jam4.apply(lambda row: row['Hydrated Lime'] + row['Hydrated_Lime_Pow_T'], axis=1)
jam4['Aluminium_Sulphate'] = jam4.apply(lambda row: row['Liquid Aluminium Sulphate'] + row['Kibbled Aluminium Sulphate'], axis=1)
jam4['Sodium_Silico_Fluoride'] = jam4.apply(lambda row: row['Sodium Silico Fluoride'] + row['Sodium Silico Fluoride (mg/L)'], axis=1)

jam4 = jam4.drop(['Sodium Aluminosilicate','Flopam FA 920 PWG','Raw Water Polyelectrolyte 4290 PWG / 4190 PWG / AN910 (mg/L)','Flopam FO 4290 PWG','Sodium_Silico_Fluoride','Microsand','Sodium Silico Fluoride (mg/L)','Sodium Silico Fluoride','Kibbled Aluminium Sulphate','Liquid Aluminium Sulphate','Hydrated Lime','Hydrated_Lime_Pow_T','Soda_Ash_T','Soda Ash','Liquid_Chlorine_T','Liquid Chlorine','Liquid Polyaluminium Chloride (mg/L)','Liquid Polyaluminium Chloride'],axis = 1)

KeyError: 'Liquid_Chlorine_T'

In [165]:
## Feature Engineering (Daily % Changes compared to median value)
the_name = jam4.drop(['wtp', 'Date','Month'],axis=1).columns
for i in the_name:
    means = jam4.groupby('wtp')[f'{i}'].transform('median')
    jam4[f'{i}_chpctm'] = (jam4[f'{i}'] - means)/means*100

In [166]:
jam4 = jam4.drop(['pH_avg', 'Turbidity_avg', 'Color_avg', 'Iron_avg',
       'Ammonia_avg', 'Manganese_avg', 'Month', 'Soda_Ash', 'Liquid_Chlorine',
       'Liquid_Polyaluminium_Chloride', 'Hydrated_Lime', 'Aluminium_Sulphate'],axis = 1)

In [167]:
cols_to_change = [col for col in jam4.columns if '_chpctm' in col]

# create a dictionary of old column names to new column names
new_cols = {col: col.replace('_chpctm', '') for col in cols_to_change}

# rename the columns in the DataFrame
jam4 = jam4.rename(columns=new_cols)
jam4 = jam4.replace([np.inf], 9999 )
jam4 = jam4.replace([-np.inf], -9999 )

In [168]:
jam4.to_csv('CP_data\ChemPare_SgSel_3.csv')

In [109]:
tfi = [x for x in jam4.columns if "_chpctm" in x]
to_c4 = jam4[tfi].copy()

## Dropping Auxiliary Chemica, Polyelectrolyte & Flouridation because project does not focus study on these product (Low usage & only certain WTP using it)
#to_c4 = to_c4.drop(['Auxiliary Chemical_chpctm', 'Polyelectrolyte_chpctm','Flouridation_chpctm'],axis=1)

In [151]:
jam4

,wtp,Date,pH_avg,Turbidity_avg,Color_avg,Iron_avg,Ammonia_avg,Manganese_avg,Soda_Ash,Liquid_Chlorine,Liquid_Polyaluminium_Chloride,Hydrated_Lime,Aluminium_Sulphate
0,Batang Kali,2021-04-01,0.594354,246.385542,1610.344828,30.000000,180.0,0.000000,0.961538,5.309735,NaN,NaN,18.533944
1,Batang Kali,2021-04-02,-1.188707,-31.686747,44.827586,10.000000,20.0,0.000000,-2.884615,4.424779,NaN,NaN,11.037068
2,Batang Kali,2021-04-03,-1.783061,77.349398,44.827586,-35.000000,120.0,0.000000,-5.769231,0.000000,NaN,NaN,37.192836
3,Batang Kali,2021-04-04,-1.931649,14.457831,1079.310345,10.000000,370.0,100.000000,-0.961538,4.424779,NaN,NaN,35.943357
4,Batang Kali,2021-04-05,-1.783061,8.554217,-41.379310,-30.000000,50.0,0.000000,0.961538,3.982301,NaN,NaN,42.357351
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999,Sungai Selangor Fasa 3,2021-09-26,-2.426343,178.072740,2.196721,89.808917,70.0,109.090909,NaN,2.993295,20.592076,-8.614187,NaN
3000,Sungai Selangor Fasa 3,2021-09-27,-4.332756,129.598877,4.655738,68.152866,50.0,150.000000,NaN,11.034140,34.351546,-6.527459,NaN
3001,Sungai Selangor Fasa 3,2021-09-28,-3.986135,125.026313,5.737705,20.382166,100.0,72.727273,NaN,-14.172885,19.341397,-7.414174,NaN
3002,Sungai Selangor Fasa 3,2021-09-29,-3.986135,208.057537,-14.754098,67.515924,140.0,159.090909,NaN,-8.727700,32.849121,-6.371021,NaN


In [102]:
jam4.columns

Index(['wtp', 'pH_avg', 'Date', 'Turbidity_avg', 'Color_avg', 'Iron_avg',
       'Ammonia_avg', 'Manganese_avg', 'Month', 'Soda_Ash', 'Liquid_Chlorine',
       'Liquid_Polyaluminium_Chloride', 'Hydrated_Lime', 'Aluminium_Sulphate',
       'pH_avg_chpctm', 'Turbidity_avg_chpctm', 'Color_avg_chpctm',
       'Iron_avg_chpctm', 'Ammonia_avg_chpctm', 'Manganese_avg_chpctm',
       'Soda_Ash_chpctm', 'Liquid_Chlorine_chpctm',
       'Liquid_Polyaluminium_Chloride_chpctm', 'Hydrated_Lime_chpctm',
       'Aluminium_Sulphate_chpctm'],
      dtype='object')

In [111]:
## General Columns for EDA
ge = ['pH_avg','Turbidity_avg', 'Color_avg','Iron_avg', 'Ammonia_avg', 'Manganese_avg','Soda_Ash', 'Liquid_Chlorine',
       'Liquid_Polyaluminium_Chloride', 'Hydrated_Lime', 'Aluminium_Sulphate']
lib = list(to_c4.columns) + ['Date','wtp'] + ge
to_c6 = FFDB[lib].copy()

KeyError: "['pH_avg_chpctm', 'Turbidity_avg_chpctm', 'Color_avg_chpctm', 'Iron_avg_chpctm', 'Ammonia_avg_chpctm', 'Manganese_avg_chpctm', 'Soda_Ash_chpctm', 'Liquid_Chlorine_chpctm', 'Liquid_Polyaluminium_Chloride_chpctm', 'Hydrated_Lime_chpctm', 'Aluminium_Sulphate_chpctm', 'Soda_Ash', 'Liquid_Chlorine', 'Liquid_Polyaluminium_Chloride', 'Hydrated_Lime', 'Aluminium_Sulphate'] not in index"

In [79]:
to_c6.to_csv('C_data/WGIDB_SgSel.csv')